You need to log into your questrade account(s) and generate a refresh token.

Once you have done it, paste it below. You only need to do this once because you will receive a new refresh token each time you redeem it for an access token, and this script will automatically save your new tokens for you

Make sure you do not accidentally share the token file

In [ ]:
from QuestradeToken import Token
Token.Save(['YourRefreshToken1', 'YourRefreshToken2'])

In [ ]:
import pandas as pd
import numpy as np
from QuestradeDatasource import QuestradeDatasource
from QuestradeDatasource import QuestradeDatasourceManager
import QuestradeConfig as cfg
import asyncio
import requests
import time
import datetime
from FuncUtils import *
from functools import partial

class BalanceActor:
    def __init__(self):
        self._loop = asyncio.get_event_loop()
        self._values = []
        self.df = pd.DataFrame()

    def Start(self):
        self._loop.run_until_complete(self._run())
    
    async def _run(self):
        target_allocation = pd.DataFrame(
            cfg.target_allocation,
            columns=['category','targetAllocation'])

        def Handler(x):
            self.df = pd.concat([self.df, x])
            
        target_allocation = pd.DataFrame(
            cfg.target_allocation,
            columns=['category','targetAllocation'])
        
        with QuestradeDatasourceManager() as ds:
            print("Time now: ", datetime.datetime.now())
            await AsyncRunAll(self._loop, [MakeFunc(aDs.GetPositions, Handler) for aDs in ds])
            print("Time now: ", datetime.datetime.now())
            df = self.df
            t1 = df.groupby(['time','category']).agg({'currentMarketValue' : np.sum}).reset_index()
            temp = t1
            temp['actual'] = temp['currentMarketValue']
            temp_sum = temp['actual'].sum()
            temp['actual'] = (temp['actual'] / temp_sum)
            temp = pd.merge(target_allocation, temp, on='category')
            temp['_diff'] = (temp['targetAllocation'] - temp['actual'])
            temp['rebalanceAmount'] = temp['_diff'] * temp_sum
            threshold = cfg.threshold
            temp.loc[abs(temp._diff) > threshold, 'needRebalance'] = 1
            temp.loc[abs(temp._diff) <= threshold, 'needRebalance'] = 0
            needRebalance = False
            if temp['needRebalance'].sum() > 0:
                needRebalance = True
                print("Need rebalance")
            else:
                print("Don't need rebalance")
            self.results = temp
        
a = BalanceActor()
a.Start()
a.results